In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

In [2]:
# Utils

from statistics import mean
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os, re
import xml.etree.ElementTree as ET
from datetime import datetime


def time_diff(df):
    res = []
    for i in range(len(df["Time (s)"]) - 1):
        val1 = df["Time (s)"][i]
        val2 = df["Time (s)"][i + 1]
        
        res.append(val2 - val1)
    return mean(res)

def read_phyphox(parent_dir):
    acc = pd.read_csv(os.path.join(parent_dir, "Phyphox", "Accelerometer.csv"))
    gyro = pd.read_csv(os.path.join(parent_dir, "Phyphox", "Gyroscope.csv"))
    loc = pd.read_csv(os.path.join(parent_dir, "Phyphox", "Location.csv"))
        
    data_len = min(len(acc), len(gyro))
    acc = acc[0:data_len]
    gyro = gyro[0:data_len]
        
    time_step = mean([time_diff(acc), time_diff(gyro)])
    
    for i in range(data_len):
        time = time_step * i
        acc["Time (s)"][i] = time
        gyro["Time (s)"][i] = time
    
    acc.set_index('Time (s)', inplace=True)
    gyro.set_index('Time (s)', inplace=True)
    loc.set_index('Time (s)', inplace=True)
        
            
    merged = acc.join(gyro, how="outer")
    merged = pd.concat([merged, loc]).sort_index().interpolate()
    
    # Rename columns
    merged.index.names = ["time"]
    merged.rename(inplace=True, columns={
        "Acceleration x (m/s^2)": "acceleration_x",
        "Acceleration y (m/s^2)": "acceleration_y",
        "Acceleration z (m/s^2)": "acceleration_z",
        "Gyroscope x (rad/s)": "gyroscope_x",
        "Gyroscope y (rad/s)": "gyroscope_y",
        "Gyroscope z (rad/s)": "gyroscope_z",
        "Latitude (°)": "latitude",
        "Longitude (°)": "longitude",
        "Height (m)": "height",
        "Velocity (m/s)": "velocity",
        "Direction (°)": "direction",
        "Horizontal Accuracy (m)": "h_accuracy",
        "Vertical Accuracy (m)": "v_accuracy",

    })
    merged = merged.dropna()
    
    time_df = pd.read_csv(os.path.join(parent_dir, "Phyphox", "meta", "time.csv"))
    start_time = time_df.loc[time_df["event"] == "START"]["system time"][0]
    
    merged.reset_index(inplace=True)
    merged['time'] = pd.to_datetime(merged['time'] + start_time, unit='s')
    return merged



def readtcx(parent_dir):
    # Function to parse heart rate data from .tcx files my Garmin produces
    # Returns DataFrame with 'time' and 'hr' columns
    
    heartrate_data = []    
    # iterating over all files
    for files in os.listdir(parent_dir):
        if files.endswith("tcx"):
            print(files)
            with open(os.path.join(parent_dir, files)) as xml_file:
                xml_str = xml_file.read()
                xml_str = re.sub(' xmlns="[^"]+"', '', xml_str, count=1)
                root = ET.fromstring(xml_str)
                activities = root.findall('.//Activity')
                for activity in activities:
                    tracking_points = activity.findall('.//Trackpoint')
                    for tracking_point in list(tracking_points):
                        children = list(tracking_point)
                        time = datetime.strptime(children[0].text, '%Y-%m-%dT%H:%M:%S.%fZ')
                        hr = list(tracking_point.find('HeartRateBpm'))[0].text
                        heartrate_data.append([time, float(hr)])
        else:
            continue
    
    df = pd.DataFrame(heartrate_data, columns=['start_time', 'heart_rate']).dropna()
    return df


def read_samsung_health(parent_dir):
    files = os.scandir(os.path.join(parent_dir, "SamsungHealth"))
    filename = next(filter(lambda file: "com.samsung.health.exercise.live_data.json" in file.name, files)).name
        
    df = pd.read_json(os.path.join(parent_dir, "SamsungHealth", filename))
    df = df[["start_time", "heart_rate"]].dropna()

    return df

def read_combined(parent_dir, activity_type, file_type):
    pp = read_phyphox(parent_dir)

    # if samsung data or tcx format for heart_rate
    if file_type == "json":
        sh = read_samsung_health(parent_dir)
    else:
        sh = readtcx(parent_dir)

    pp["heart_rate"] = np.nan
    
    # Finds the time intervals of the heart rate measurements and update the phyphox heart rate accordingly
    # This is done as samsung measurements are not very fine grained and only give use relatively large
    # time intervals
    for i in range(len(sh) - 1):
        row1 = sh.iloc[i]
        row2 = sh.iloc[i + 1]
        pp.loc[(pp["time"] >= row1["start_time"]) & (pp["time"] < row2["start_time"]), "heart_rate"] = row1["heart_rate"]
    df = pp.dropna()

    # add an extra column to add the activity type
    df["label"] = activity_type
    return df


In [3]:
def read_datasets(data_dir):
    res_dfs = []
    temp_df = pd.DataFrame()


    for f, act_type, file_type in data_dir:
        temp_df = read_combined(f, act_type, file_type)
        temp_df = temp_df.reset_index().drop(columns="index", errors="ignore")
        res_dfs.append(temp_df.copy())
    
    return res_dfs

In [4]:
import Python3Code.Chapter3.OutlierDetection as od

def clean_chauvenet(df, cols=["acceleration_x", "acceleration_y", "acceleration_z", "gyroscope_x", "gyroscope_y", "gyroscope_z", "latitude", "longitude", "height", "velocity", "heart_rate"]):
    n_outliers = 0
    distribution_od = od.DistributionBasedOutlierDetection()
    for col in cols:
        df[col + "_outlier"] = False
        df = distribution_od.chauvenet(df, col, 2)
        n_outliers += len(df.loc[df[col + "_outlier"] == True])    
        df.loc[df[col + "_outlier"] == True, col] = np.nan
        df.drop(columns=col + "_outlier", inplace=True)
    
    print(f"Chauvenet found {n_outliers} outliers")
    df.interpolate(inplace=True)
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns="index", errors="ignore", inplace=True)
    return df

In [138]:
data_dir = [#("Bike 07-06-23 8:52","biking","json"),
                ("Bike 07-06-23 14:27","biking","json"),
                ("Tram 08-06-23 11:05","public_transport","json"),
                ("Walk 08-06-23 10:41","walking","json"),
                ("walking1","walking","tcx"),
                ("walking2","walking","tcx"),
                ("sitting1","sitting","tcx"),
                ("sitting2","sitting","tcx")]

orig_datasets = read_datasets(data_dir)

for df in orig_datasets:
    clean_chauvenet(df)

orig_datasets[0]

<ipython-input-2-760c76ba7a1b>:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = activity_type


walking1.tcx
walking2.tcx
sitting.tcx
sitting2.tcx
Chauvenet found 255 outliers
Chauvenet found 363 outliers
Chauvenet found 64 outliers
Chauvenet found 23 outliers
Chauvenet found 67 outliers
Chauvenet found 191 outliers
Chauvenet found 390 outliers


,time,acceleration_x,acceleration_y,acceleration_z,gyroscope_x,gyroscope_y,gyroscope_z,latitude,longitude,height,velocity,direction,h_accuracy,v_accuracy,heart_rate,label
0,2023-06-07 11:56:26.677337885,3.812580,-0.154950,2.682000,0.413188,0.031212,0.012788,52.334749,4.864622,-1.688446,1.369676,169.266003,3.24,6.14,104.0,biking
1,2023-06-07 11:56:26.927304745,4.799370,-3.727950,-1.735950,0.258087,0.165137,0.646800,52.334749,4.864626,-1.748443,1.292982,164.042001,3.18,6.68,104.0,biking
2,2023-06-07 11:56:27.177271843,5.786160,-3.867000,-0.112050,-0.114813,0.458150,0.409819,52.334749,4.864630,-1.808440,1.216289,158.817999,3.12,7.22,104.0,biking
3,2023-06-07 11:56:27.427238703,6.772950,-6.589950,1.590000,-0.573788,-0.223850,0.172837,52.334750,4.864634,-1.868437,1.139595,153.593997,3.06,7.76,104.0,biking
4,2023-06-07 11:56:27.548669100,6.752475,-7.456950,0.851025,-0.328350,-0.071844,0.131312,52.334750,4.864638,-1.928434,1.062901,148.369995,3.00,8.30,104.0,biking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7483,2023-06-07 12:21:23.229163408,-1.780950,-9.523951,-2.586000,1.098350,-0.448388,-0.719262,52.278317,4.825148,-53.089947,0.000000,293.380005,2.00,6.92,151.0,biking
7484,2023-06-07 12:21:23.479130268,1.044000,-9.225000,1.839000,0.159362,0.112613,0.077688,52.278317,4.825149,-53.069946,0.000000,293.380005,2.00,7.08,151.0,biking
7485,2023-06-07 12:21:23.729097128,-1.207050,-9.472051,-1.357050,0.081400,-0.102437,-0.004262,52.278317,4.825149,-53.049946,0.000000,293.380005,2.00,7.24,151.0,biking
7486,2023-06-07 12:21:23.875818014,-0.524025,-9.304050,-0.554025,0.733425,-0.715412,-0.340313,52.278317,4.825150,-53.029946,0.000000,293.380005,2.00,7.40,151.0,biking


In [61]:
from Python3Code.Chapter4.FrequencyAbstraction import FourierTransformation
import copy

def extract_frequency(df, periodic_predictor_cols, fs=4, window_size=40):
    print("Extracting frequency features")
    FreqAbs = FourierTransformation()
    data_table = FreqAbs.abstract_frequency(df, periodic_predictor_cols, window_size, fs)
    return data_table

In [62]:
from Python3Code.Chapter4.TemporalAbstraction import NumericalAbstraction

def extract_temporal(df, feature_map, window_size=40):
    print("Extracting temporal features")
    NumAbs = NumericalAbstraction()
    
    if "time" in df:
        df.set_index("time", inplace=True)
        
    for (func, cols) in feature_map.items():
        df = NumAbs.abstract_numerical(df, cols, 40, func)
    
    df.reset_index(inplace=True)
    df.drop(columns="index", errors="ignore", inplace=True)
    return df

In [217]:
def extract_features(df, temporal_features, window_size=40, remove_overlap=0.5, frequency_features=['acceleration_x', 'acceleration_y', 'acceleration_z', 'gyroscope_x', 'gyroscope_y', 'gyroscope_z']):
    df_copy = df.copy()
    extract_frequency(df_copy, window_size=window_size, periodic_predictor_cols=frequency_features)
    extract_temporal(df_copy, temporal_features, window_size=window_size / 4)
    
    keep = []
    
    item_count = 0
    for i in range(len(df_copy)):
        if item_count == 0:
            keep.append(df_copy.iloc[[i]])
        
        item_count = (item_count + 1) % (window_size * remove_overlap + 1)
    
    print("Len before:", len(df))
    print("Kept:", len(keep))
    
    keep = pd.concat(keep)
    
    keep.replace([np.inf, -np.inf], np.nan, inplace=True)
    keep.dropna(inplace=True)
    keep.reset_index(inplace=True)
    keep.drop(columns="index", errors="ignore", inplace=True)
    return keep

In [226]:
temporal_features = {
    "std": ["acceleration_x", "acceleration_y", "acceleration_z", "gyroscope_x", "gyroscope_y", "gyroscope_z"],
    "mean": ["acceleration_x", "acceleration_y", "acceleration_z", "gyroscope_x", "gyroscope_y", "gyroscope_z", "latitude", "longitude", "height", "velocity", "heart_rate"],
    "median": ["height", "heart_rate"],
    "slope": ["latitude", "longitude", "velocity"],
    "max": ["velocity", "heart_rate"],
    "min": ["velocity", "heart_rate"],
}

datasets = []

for df in orig_datasets:
    datasets.append(extract_features(df, temporal_features, remove_overlap=0.2))

datasets[0]

Extracting frequency features
Extracting temporal features
Len before: 7488
Kept: 832
Extracting frequency features
Extracting temporal features
Len before: 5252
Kept: 584
Extracting frequency features
Extracting temporal features
Len before: 2606
Kept: 290
Extracting frequency features
Extracting temporal features
Len before: 2632
Kept: 293
Extracting frequency features
Extracting temporal features
Len before: 2425
Kept: 270
Extracting frequency features
Extracting temporal features
Len before: 3414
Kept: 380
Extracting frequency features
Extracting temporal features
Len before: 6390
Kept: 710


,time,acceleration_x,acceleration_y,acceleration_z,gyroscope_x,gyroscope_y,gyroscope_z,latitude,longitude,height,...,heart_rate_temp_mean_ws_40,height_temp_median_ws_40,heart_rate_temp_median_ws_40,latitude_temp_slope_ws_40,longitude_temp_slope_ws_40,velocity_temp_slope_ws_40,velocity_temp_max_ws_40,heart_rate_temp_max_ws_40,velocity_temp_min_ws_40,heart_rate_temp_min_ws_40
0,2023-06-07 11:56:35.876088142,-1.42350,-7.264500,-6.405975,-1.125781,-0.309650,0.057269,52.334937,4.864738,-0.828115,...,103.913043,-1.174847,104.0,0.000005,2.601916e-06,0.001696,1.369676,104.0,0.000000,103.0
1,2023-06-07 11:56:37.675881624,-0.78705,-4.837050,-9.052951,0.994125,0.574337,0.139838,52.334937,4.864720,-0.403124,...,103.763636,-1.108124,104.0,0.000004,2.134242e-06,0.003670,1.369676,104.0,0.000000,103.0
2,2023-06-07 11:56:39.425649881,1.29300,-8.317050,-8.142000,-1.055863,-0.355713,0.094875,52.334931,4.864691,-0.330751,...,103.656250,-1.028181,104.0,0.000003,1.543498e-06,0.003458,2.278778,104.0,0.000000,103.0
3,2023-06-07 11:56:41.175418139,-0.22500,-1.840950,-8.475000,2.551038,0.876288,0.011825,52.334910,4.864587,-0.333705,...,103.575342,-0.961532,104.0,0.000003,4.993461e-07,0.025972,3.594408,104.0,0.000000,103.0
4,2023-06-07 11:56:42.925186396,2.00700,-7.903050,-8.446950,-1.553887,-0.265650,-0.322163,52.334909,4.864417,-0.336659,...,103.512195,-0.901486,104.0,0.000002,-1.049591e-06,0.031300,3.681808,104.0,0.000000,103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,2023-06-07 12:21:15.230222464,0.50100,-2.121000,-9.579000,-0.010588,0.175588,0.272662,52.278390,4.825317,-53.709749,...,150.771144,-50.229484,151.0,0.000003,-1.242995e-05,-0.008424,5.108618,151.0,2.970462,150.0
823,2023-06-07 12:21:16.979990721,-0.41895,-5.139000,-7.780951,0.010450,0.085662,0.134613,52.278342,4.825271,-54.049842,...,150.815920,-50.229484,151.0,0.000003,-1.210172e-05,-0.009250,5.108618,151.0,2.970462,150.0
824,2023-06-07 12:21:18.885046005,0.91995,-5.232525,-7.313475,0.285381,0.280569,0.329656,52.278310,4.825220,-54.329916,...,150.860697,-50.229484,151.0,0.000002,-1.173127e-05,-0.010697,5.108618,151.0,1.947194,150.0
825,2023-06-07 12:21:20.729494333,-3.78795,-8.317050,-5.164950,-0.215600,0.457600,-0.194287,52.278312,4.825166,-53.749943,...,150.905473,-50.269361,151.0,0.000002,-1.133452e-05,-0.013402,5.077735,151.0,1.393662,150.0


In [307]:
features = [
    # Freq
    "acceleration_x_max_freq", "acceleration_x_freq_weighted", #"acceleration_x_pse",
    "acceleration_y_max_freq", "acceleration_y_freq_weighted", #"acceleration_y_pse",
    "acceleration_z_max_freq", "acceleration_z_freq_weighted", #"acceleration_z_pse",
    "gyroscope_x_max_freq", "gyroscope_x_freq_weighted", #"gyroscope_x_pse",
    "gyroscope_y_max_freq", "gyroscope_y_freq_weighted", #"gyroscope_y_pse",
    "gyroscope_z_max_freq", "gyroscope_z_freq_weighted", #"gyroscope_z_pse",
    
    # Temp
    
    # std
    "acceleration_x_temp_std_ws_40", "acceleration_y_temp_std_ws_40", "acceleration_z_temp_std_ws_40",
    "gyroscope_x_temp_std_ws_40", "gyroscope_y_temp_std_ws_40", "gyroscope_z_temp_std_ws_40",
    
    # mean
    #"acceleration_x_temp_mean_ws_40", "acceleration_y_temp_mean_ws_40", "acceleration_z_temp_mean_ws_40",
    #"gyroscope_x_temp_mean_ws_40", "gyroscope_y_temp_mean_ws_40", "gyroscope_z_temp_mean_ws_40",
    #"latitude_temp_mean_ws_40", "longitude_temp_mean_ws_40", "height_temp_mean_ws_40",
    #"velocity_temp_mean_ws_40", "heart_rate_temp_mean_ws_40",
    
    # median
    #"height_temp_median_ws_40", "heart_rate_temp_median_ws_40",
    
    # slope
    "latitude_temp_slope_ws_40", "longitude_temp_slope_ws_40", #"velocity_temp_slope_ws_40",
    
    # max
    "velocity_temp_max_ws_40", "heart_rate_temp_max_ws_40"
    
    # min
    "velocity_temp_min_ws_40", "heart_rate_temp_min_ws_40"
]

#features = [
#    "heart_rate", "heart_rate_temp_max_ws_40", "heart_rate_temp_min_ws_40",
#    
#    
#]

features = ['heart_rate',
 'acceleration_x_max_freq',
 'acceleration_z_max_freq',
 'gyroscope_x_temp_std_ws_40',
 'acceleration_y_temp_mean_ws_40',
 'heart_rate_temp_mean_ws_40',
 'heart_rate_temp_median_ws_40',
 'velocity_temp_max_ws_40',
 'heart_rate_temp_max_ws_40',
 'heart_rate_temp_min_ws_40']

print(sum(map(lambda x: len(x), datasets)))
print(len(features))

3324
10


In [329]:
def train_test_split(df, train_percentage=0.8):
    return np.split(df, [int(train_percentage * len(df))])

train_datasets = []
test_datasets = []

train_len = 0
test_len = 0

for df in datasets:
    train, test = train_test_split(df, train_percentage=0.6)
    train_len += len(train)
    test_len += len(test)
    train_datasets.append(train)
    test_datasets.append(test)

print(f"train len: {train_len}")
print(f"test  len: {test_len}")

train len: 1993
test  len: 1331


In [330]:
def fold_merge(datasets, kfolds=10):
    datasets_folds = []
    for df in datasets:
        datasets_folds.append(np.array_split(df, kfolds))
    
    ordered_folds = []
    for i in range(kfolds):
        folds = []
        for j in range(len(datasets_folds)):
            folds.append(datasets_folds[j][i])
        
        ordered_folds.append(pd.concat(folds, axis=0))
        ordered_folds[i].set_index("time", inplace=True)
    
    #df = pd.concat(ordered_folds, axis=0)
    #df.reset_index(inplace=True)
    return ordered_folds

folds = fold_merge(train_datasets)

for f in folds:
    print(df.isnull().values.any())

folds[0]

False
False
False
False
False
False
False
False
False
False


,acceleration_x,acceleration_y,acceleration_z,gyroscope_x,gyroscope_y,gyroscope_z,latitude,longitude,height,velocity,...,heart_rate_temp_mean_ws_40,height_temp_median_ws_40,heart_rate_temp_median_ws_40,latitude_temp_slope_ws_40,longitude_temp_slope_ws_40,velocity_temp_slope_ws_40,velocity_temp_max_ws_40,heart_rate_temp_max_ws_40,velocity_temp_min_ws_40,heart_rate_temp_min_ws_40
time,,,,,,,,,,,,,,,,,,,,,
2023-06-07 11:56:35.876088142,-1.423500,-7.264500,-6.405975,-1.125781,-0.309650,0.057269,52.334937,4.864738,-0.828115,1.093270,...,103.913043,-1.174847,104.0,5.355397e-06,2.601916e-06,0.001696,1.369676,104.0,0.0,103.0
2023-06-07 11:56:37.675881624,-0.787050,-4.837050,-9.052951,0.994125,0.574337,0.139838,52.334937,4.864720,-0.403124,0.282068,...,103.763636,-1.108124,104.0,4.284516e-06,2.134242e-06,0.003670,1.369676,104.0,0.0,103.0
2023-06-07 11:56:39.425649881,1.293000,-8.317050,-8.142000,-1.055863,-0.355713,0.094875,52.334931,4.864691,-0.330751,2.278778,...,103.656250,-1.028181,104.0,3.466938e-06,1.543498e-06,0.003458,2.278778,104.0,0.0,103.0
2023-06-07 11:56:41.175418139,-0.225000,-1.840950,-8.475000,2.551038,0.876288,0.011825,52.334910,4.864587,-0.333705,3.594408,...,103.575342,-0.961532,104.0,2.663886e-06,4.993461e-07,0.025972,3.594408,104.0,0.0,103.0
2023-06-07 11:56:42.925186396,2.007000,-7.903050,-8.446950,-1.553887,-0.265650,-0.322163,52.334909,4.864417,-0.336659,2.107891,...,103.512195,-0.901486,104.0,2.044129e-06,-1.049591e-06,0.031300,3.681808,104.0,0.0,103.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-11 11:20:41.393084049,0.009571,0.014955,9.827323,-0.000000,-0.000305,0.000153,52.321532,4.871901,6.057116,0.000000,...,55.091463,6.057115,55.0,-2.483007e-08,-4.949992e-08,0.000000,0.000000,56.0,0.0,53.0
2023-06-11 11:20:43.619078159,0.009571,0.015553,9.823733,0.000000,0.000153,0.000000,52.321533,4.871901,6.057117,0.000000,...,55.085366,6.057115,55.0,-1.670386e-08,-4.376656e-08,0.000000,0.000000,56.0,0.0,53.0
2023-06-11 11:20:45.845072508,0.012562,0.016152,9.827323,-0.000305,-0.000000,0.000153,52.321533,4.871902,6.057118,0.000000,...,55.030488,6.057115,55.0,-8.577659e-09,-3.711667e-08,0.000000,0.000000,56.0,0.0,53.0


In [331]:
# Obtain a dataframe X containing only the selected features and Y containing only the labels
def separate_XY(df, cols, label_col="label"):
    return df[df.columns.intersection(cols)], df[label_col]

In [358]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import sys
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from statistics import mean

def cross_validation(model, folds, features):
    val_scores = []
    for fold_nr in range(len(folds) - 1):
        df_train = pd.concat(folds[0:fold_nr + 1])
        X_train, y_train = separate_XY(df_train, features)
        
        model.fit(X_train, y_train)
        
        df_val = folds[fold_nr + 1]
        X_val, y_val = separate_XY(df_val, features)
        
        y_train_pred, y_val_pred = model.predict(X_train), model.predict(X_val)
        #print(y_train_pred)
        #print(y_train_pred, y_val_pred)
        
        #cm = confusion_matrix(y_val, y_val_pred)
        #print(cm)

        train_f1, val_f1 = f1_score(y_train, y_train_pred, average="macro"), f1_score(y_val, y_val_pred, average="macro")
        print(f"Fold {fold_nr}, train F1-score:{train_f1}, validation F1-score: {val_f1}")
        val_scores.append(val_f1)
    
    print(f"{len(folds)}-fold cross validation mean F1-score: {mean(val_scores)}")
    
    



np.set_printoptions(threshold=sys.maxsize)

classifier = GaussianNB()
#classifier = RandomForestClassifier(n_estimators=10, min_samples_leaf=5, criterion='gini')
cross_validation(classifier, folds, features)


# Test
X_train, y_train = separate_XY(pd.concat(train_datasets), features)

classifier.fit(X_train, y_train)

X_test, y_test = separate_XY(pd.concat(test_datasets), features)
y_test_pred = classifier.predict(X_test)
test_f1 = f1_score(y_test, y_test_pred, average="macro")
print(f"Test F1-score: {test_f1}")



Fold 0, train F1-score:1.0, validation F1-score: 0.8669467787114846
Fold 1, train F1-score:0.9975359891243658, validation F1-score: 1.0
Fold 2, train F1-score:0.9983495473511051, validation F1-score: 1.0
Fold 3, train F1-score:0.998480501255821, validation F1-score: 1.0
Fold 4, train F1-score:0.9987846656257685, validation F1-score: 0.9877450980392157
Fold 5, train F1-score:1.0, validation F1-score: 0.9420867167151831
Fold 6, train F1-score:0.9992877131533608, validation F1-score: 1.0
Fold 7, train F1-score:0.9993757452479484, validation F1-score: 1.0
Fold 8, train F1-score:0.9994444108347296, validation F1-score: 1.0
10-fold cross validation mean F1-score: 0.9774198437184315
Test F1-score: 0.9917017408820687


In [305]:
from sklearn.model_selection import StratifiedKFold

def train(fold_nr, classifier, X_train, y_train, X_val, y_val):
    """
    Train the model and return validation score
    """
    model = classifier
    model.fit(X_train, y_train)
    
    y_train_pred, y_val_pred = model.predict(X_train), model.predict(X_val)
    train_f1, val_f1 = f1_score(y_train, y_train_pred, average="macro"), f1_score(y_val, y_val_pred, average="macro")
    
    print(f"Fold {fold_nr}, train F1-score:{train_f1}, validation F1-score: {val_f1}")
    return train_f1, val_f1

df = pd.concat(datasets)
X, y = separate_XY(df, features)

skf = StratifiedKFold(n_splits=10, shuffle=False)
for fold_nr, (train_index, val_index) in enumerate(skf.split(X, y)):
    train_score, val_score = train(fold_nr, 
                                     classifier, 
                                     X_train=X.iloc[train_index], 
                                     y_train=y.iloc[train_index], 
                                     X_val=X.iloc[val_index], 
                                     y_val=y.iloc[val_index])

Fold 0, train F1-score:1.0, validation F1-score: 0.9850105913187089
Fold 1, train F1-score:0.9993324004171383, validation F1-score: 1.0
Fold 2, train F1-score:0.9993324004171383, validation F1-score: 1.0
Fold 3, train F1-score:0.9989243492566026, validation F1-score: 0.9699872286079183
Fold 4, train F1-score:0.9989247872537292, validation F1-score: 1.0
Fold 5, train F1-score:0.9989247872537292, validation F1-score: 1.0
Fold 6, train F1-score:0.9992586126710289, validation F1-score: 1.0
Fold 7, train F1-score:0.9992586032009529, validation F1-score: 1.0
Fold 8, train F1-score:0.999332852666809, validation F1-score: 1.0
Fold 9, train F1-score:0.9988505400296974, validation F1-score: 0.9541083916083917


In [306]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

df = pd.concat(train_datasets)

all_features = df.columns.values.tolist()
all_features.remove("label")
all_features.remove("time")
all_features.remove("v_accuracy")
all_features.remove("h_accuracy")
all_features.remove("direction")




X, y = separate_XY(df, all_features)

sel = SelectKBest(f_classif, k=10)
sel.fit_transform(X, y)
mask = sel.get_support() #list of booleans
new_features = [] # The list of your K best features

for bool_val, feature in zip(mask, all_features):
    if bool_val:
        new_features.append(feature)
        
new_features

['heart_rate',
 'acceleration_x_max_freq',
 'acceleration_z_max_freq',
 'gyroscope_x_temp_std_ws_40',
 'acceleration_y_temp_mean_ws_40',
 'heart_rate_temp_mean_ws_40',
 'heart_rate_temp_median_ws_40',
 'velocity_temp_max_ws_40',
 'heart_rate_temp_max_ws_40',
 'heart_rate_temp_min_ws_40']